In [5]:
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam
from keras.layers import Activation, Dense, Flatten, Input, GlobalAveragePooling2D

In [6]:
pixels = 200
IMAGE_SIZE = (pixels, pixels)
print(f"Input size {IMAGE_SIZE}")

BATCH_SIZE = 16#@param {type:"integer"}

normalization_layer = tf.keras.layers.Rescaling(1. / 255)
preprocessing_model = tf.keras.Sequential([normalization_layer])


data_dir = 'D:/kolya/4th year/GP/New Idea/GitHub Codes/Sign-Language-Translator-main/ASL Alphabitic/asl_alphabet_train/asl_alphabet_train'
def build_dataset(subset):
  return tf.keras.preprocessing.image_dataset_from_directory(
      data_dir,
      validation_split=.20,
      subset=subset,
      label_mode="categorical",
      # Seed needs to provided when using validation_split and shuffle = True.
      # A fixed seed is used so that the validation set is stable across runs.
      seed=123,
      image_size=IMAGE_SIZE,
      batch_size=1)

train_ds = build_dataset("training")
class_names = tuple(train_ds.class_names)
train_size = train_ds.cardinality().numpy()
train_ds = train_ds.unbatch().batch(BATCH_SIZE)
train_ds = train_ds.repeat()   #hna a3tkd kan mafroud n7ot gwa el repeat el rkm el epoch kol el sites bt3ml kda

train_ds = train_ds.map(lambda images, labels:
                        (preprocessing_model(images), labels))

val_ds = build_dataset("validation")
valid_size = val_ds.cardinality().numpy()
val_ds = val_ds.unbatch().batch(BATCH_SIZE)
val_ds = val_ds.map(lambda images, labels:
                    (normalization_layer(images), labels))

Input size (200, 200)
Found 87000 files belonging to 29 classes.
Using 69600 files for training.
Found 87000 files belonging to 29 classes.
Using 17400 files for validation.


In [7]:
NUM_OF_CLASSES = 29

In [8]:
mobile = tf.keras.applications.mobilenet.MobileNet(input_tensor=Input(shape=(200,200,3)))
mobile.summary()

Model: "mobilenet_1.00_224"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 200, 200, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 100, 100, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 100, 100, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 100, 100, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 100, 100, 32)     288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 100, 100, 32)     128       
 ation)                                         

In [9]:
x = mobile.layers[-9].output
Flatten_Layer = GlobalAveragePooling2D()(x)
output = Dense(units= NUM_OF_CLASSES, kernel_regularizer=tf.keras.regularizers.l2(0.0001), activation = 'softmax')(Flatten_Layer)
model = Model(inputs = mobile.input, outputs= output)
for layer in model.layers[:-26] :
    layer.trainable = False
model.build((None,)+IMAGE_SIZE+(3,))
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 200, 200, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 100, 100, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 100, 100, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 100, 100, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 100, 100, 32)     288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 100, 100, 32)     128       
 ation)                                                      

In [10]:
model.compile(optimizer= Adam(learning_rate= 0.0001), loss = 'categorical_crossentropy', metrics=['accuracy'])

In [11]:
steps_per_epoch = train_size  // BATCH_SIZE
validation_steps = valid_size // BATCH_SIZE

model.fit(x = train_ds, validation_data=val_ds, epochs=2, verbose=1, steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps)

Epoch 1/2
4350/4350 [==============================] - 890s 200ms/step - loss: 0.2013 - accuracy: 0.9727 - val_loss: 0.0099 - val_accuracy: 0.9999
Epoch 2/2
4350/4350 [==============================] - 1137s 261ms/step - loss: 0.0110 - accuracy: 0.9998 - val_loss: 0.0079 - val_accuracy: 0.9997


In [12]:
saved_model_path = "Saved_Model_With_TF1"
tf.saved_model.save(model, saved_model_path)
model.save("MobileNetWithoutTheLast6Layers.h5")
print("Saved Model")

INFO:tensorflow:Assets written to: Saved_Model_With_TF1\assets


INFO:tensorflow:Assets written to: Saved_Model_With_TF1\assets


Saved Model


In [10]:
saved_model_path = "Saved_Model_With_TFWith_Keras"
tf.keras.models.save_model(model, saved_model_path)

INFO:tensorflow:Assets written to: Saved_Model_With_TFWith_Keras\assets


INFO:tensorflow:Assets written to: Saved_Model_With_TFWith_Keras\assets
